In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train =pd.read_csv('/kaggle/input/house-price-prediction-challenge/train.csv')
train.head()

In [ ]:
train.shape

No missing values

In [ ]:
train.info()

most of houses not under construction

In [ ]:
train['UNDER_CONSTRUCTION'].value_counts()

In [ ]:
train['READY_TO_MOVE'].value_counts()

In [ ]:
train['RESALE'].value_counts()

In [ ]:
train['POSTED_BY'].value_counts()

handling categorical values , there is many ways like this two ways or OHE or LE or dummies

In [ ]:
#train['POSTED']=train['POSTED_BY'].apply(lambda x :0 if x=='Dealer' else 1 if x=='Owner' else 2)

In [ ]:
#train.drop("POSTED_BY",axis=1,inplace=True)

In [ ]:
mappp={'Dealer':0,"Owner":1,'Builder':2}

train['POSTED_BY']=train['POSTED_BY'].map(mappp)

In [ ]:
train['BHK_OR_RK'].value_counts()

In [ ]:
mapp={'BHK':0,"RK":1}
train['BHK_OR_RK']=train['BHK_OR_RK'].map(mapp)

In [ ]:
train.head()

number of rooms between 1 and 20 






In [ ]:
train.plot(kind='scatter',x='TARGET(PRICE_IN_LACS)',y='BHK_NO.')

latitude between 60 and 150, doesn't highly effect

In [ ]:
train.plot(kind='scatter',x='TARGET(PRICE_IN_LACS)',y='LATITUDE')

longitude between 0 and 40 ,  doesn't highly effect but near to 40 less cost

In [ ]:
train.plot(kind='scatter',x='TARGET(PRICE_IN_LACS)',y='LONGITUDE')

log for area

In [ ]:
train['AREA']=np.log(train['SQUARE_FT'])


price direct proportional with area

In [ ]:
sns.scatterplot(data=train,x='TARGET(PRICE_IN_LACS)',y='AREA')

drop column before log

In [ ]:
train.drop('SQUARE_FT',axis=1,inplace=True)

In [ ]:
train.describe()

location doesn't highly effect to price

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(data=train,x='LONGITUDE',y='LATITUDE',hue='TARGET(PRICE_IN_LACS)')

divide features to categorical and numerical

In [ ]:
cat_features = ['POSTED_BY', 'BHK_OR_RK']
num_features = ['UNDER_CONSTRUCTION', 'RERA', 'BHK_NO.', 'READY_TO_MOVE', 'RESALE', 'LONGITUDE', 'LATITUDE', 'AREA']

area, number of rooms, price highly correlated with each other

In [ ]:
plt.figure(figsize=(10, 10))
corr=train[cat_features+num_features+['TARGET(PRICE_IN_LACS)']].corr(method='spearman')
sns.heatmap(corr, annot=True)


handling outliers

In [ ]:
outlier_percentage = {}
for feature in ['AREA', 'BHK_NO.', 'TARGET(PRICE_IN_LACS)']:
    tempData = train.sort_values(by=feature)[feature]
    Q1, Q3 = tempData.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    Lower_range = Q1 - (1.5 * IQR)
    Upper_range = Q3 + (1.5 * IQR)
    outlier_percentage[feature] = round((((tempData<(Q1 - 1.5 * IQR)) | (tempData>(Q3 + 1.5 * IQR))).sum()/tempData.shape[0])*100,2)
outlier_percentage

In [ ]:
outlier = train[(train[feature]>Lower_range) & (train[feature]<Upper_range)].reset_index(drop=True)


In [ ]:
train.drop(['ADDRESS'],axis=1,inplace=True)

In [ ]:
X=train.drop(['TARGET(PRICE_IN_LACS)'],axis=1)

y=train['TARGET(PRICE_IN_LACS)']

feature selection


under constrauction and ready to move exactly the same , location has small effect

In [ ]:
from sklearn.feature_selection import SelectKBest,f_regression
best=SelectKBest(score_func=f_regression,k='all')
fit=best.fit(X,y)
dfScores=pd.DataFrame(fit.scores_)
dfCol=pd.DataFrame(X.columns)
featScore=pd.concat([dfCol,dfScores],axis=1)
featScore.columns=['Feature','Score']
featScore=featScore.sort_values(by='Score',ascending=False).reset_index(drop=True)
print(featScore.nlargest(10,'Score'))

another way for feature selection

In [ ]:
#from sklearn.ensemble import ExtraTreesRegressor
#ex=ExtraTreesRegressor()
#ex.fit(X,y)

In [ ]:
#ex.feature_importances_


In [ ]:
#plt.figure(figsize=(10,10))
#plt.title('Feature importances')
#feat=pd.Series(ex.feature_importances_,index=X.columns)
#feat.nlargest(10).plot(kind='barh', color="r", align="center")
#plt.tight_layout()
#plt.show()

removing features with vif , there is another way called RFE "Recursive Feature Elimination"

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
v = pd.DataFrame()
v["variables"] = [feature for feature in cat_features+num_features if feature not in ['READY_TO_MOVE', 
                                                                    'RESALE', 'LATITUDE', 'LONGITUDE']]
v["VIF"] = [variance_inflation_factor(train[v['variables']].values, i) for i in range(len(v["variables"]))]
print(v)

splitting the data

In [ ]:
from sklearn.model_selection import train_test_split ,cross_val_score,RandomizedSearchCV
X_train ,X_test,y_train ,y_test =train_test_split(X,y,test_size =.2 , random_state=42)

feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression ,SGDRegressor,Lasso,Ridge,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,VotingRegressor
from xgboost.sklearn import XGBRegressor

train models

In [ ]:
models = {'Linear Regression': LinearRegression(),
         'Lasso': Lasso(),
         'Decision Tree': DecisionTreeRegressor(),
         'Random Forest': RandomForestRegressor(),
         'Gradient Boosting =':GradientBoostingRegressor(),
         'Ridge': Ridge(),
         'Stochastic Gradien Descent': SGDRegressor(),
         'Elastic': ElasticNet(),
         'xgb Regressor': XGBRegressor()}

fit and score

In [ ]:
def fit_score(models, X_train, X_test, y_train, y_test):
    np.random.seed(42)
    model_scores = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        model_scores[name] = cross_val_score(model,
                                             X_test,
                                             y_test,
                                            scoring='neg_mean_squared_error',
                                            cv=3
                                            ).mean()

    return model_scores

Decision tree has lowest mse

In [ ]:
model_scores = fit_score(models,X_train,X_test,y_train,y_test)

model_scores

voting

In [ ]:
vot=VotingRegressor([('LinearRegression',LinearRegression()),('DecisionTrees',DecisionTreeRegressor()),('LassoRegression',Lasso()),
                     ('RandomForest',RandomForestRegressor()),('ElasticNet',ElasticNet()),
                     ('StochasticGradientDescent',SGDRegressor()),
                    ('GrafientBoosting',GradientBoostingRegressor()),('Ridge',Ridge()),('xgb',XGBRegressor())])
vot.fit(X_train,y_train)
y_pred=vot.predict(X_test)

as expected , voting has smallest mse

In [ ]:
mean_squared_error(y_test,y_pred)


In [ ]:
np.random.seed(42)
params={
        'criterion':['mse', 'mae'],
         'max_features':['auto','sqrt','log2'],
        'max_depth':[2,3,10]
       }
rs = RandomizedSearchCV(DecisionTreeRegressor(),
                                param_distributions=params,
                                cv=3,
                                n_iter=30,
                                verbose=0,
                                n_jobs=-1
                               )

rs.fit(X_train, y_train)

rs.best_params_

In [ ]:
rs.best_estimator_



In [ ]:
rs.best_score_


In [ ]:
rs.score(X_test, y_test)


In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
r2_score(y_pred,y_test)

In [ ]:
test =pd.read_csv('/kaggle/input/house-price-prediction-challenge/test.csv')


In [ ]:
test.shape

In [ ]:
final=test.drop('ADDRESS',axis=1,inplace=True)

In [ ]:

mappp={'Dealer':0,"Owner":1,'Builder':2}

test['POSTED_BY']=test['POSTED_BY'].map(mappp)

In [ ]:
mapp={'BHK':0,"RK":1}
test['BHK_OR_RK']=test['BHK_OR_RK'].map(mapp)

In [ ]:
test['AREA']=np.log(test['SQUARE_FT'])


In [ ]:
test.drop('SQUARE_FT',axis=1,inplace=True)

In [ ]:
scaler=StandardScaler()
test=scaler.fit_transform(test)

In [ ]:
price=model.predict(test)

In [ ]:
result=pd.DataFrame(price,columns=["TARGET(PRICE_IN_LACS"])

result.to_csv("ResultHousePrices.csv",index=None)